In [1]:
!pip install --upgrade openai

In [2]:
import os
import openai
import glob
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
def debug_code_with_gpt(description, code_content, API_KEY, few_shot, CoT):
    if few_shot == '0' and CoT == 'n':
        # for the prompt, use prompts/zeroShot_noCoT.txt as the prompt
        with open('prompts/zeroShot_noCoT.txt', 'r') as file:
            prompt = file.read().replace('{description}', description).replace('{code_content}', code_content)
    elif few_shot == '0' and CoT == 'c':
        with open('prompts/zeroShot_CoT.txt', 'r') as file:
            prompt = file.read().replace('{description}', description).replace('{code_content}', code_content)
    elif few_shot == 'f' and CoT == 'n': 
        with open('prompts/fewShot_noCoT.txt', 'r') as file:
            prompt = file.read().replace('{description}', description).replace('{code_content}', code_content)
    elif few_shot == 'f' and CoT == 'c':
        with open('prompts/fewShot_CoT.txt', 'r') as file:
            prompt = file.read().replace('{description}', description).replace('{code_content}', code_content)
    else:
        raise ValueError("few_shot should be '0' or 'f', CoT should be 'n' or 'c'.")

    try:
        client = openai.OpenAI(api_key=API_KEY)
#         # If we are about to hit the token limit, just wait.
#         # Check the token limit, and wait if it is less than the token required for the next API call
#         # Guess the token required for the next API call
#         token_required = 2 * len(prompt) + 1000
#         token_remaining = int(client.usage().data['data']['remaining'])
#         if token_remaining < token_required:
#             print("Waiting for token reset...")
#             while token_remaining < token_required:
#                 token_remaining = int(client.usage().data['data']['remaining'])
#                 time.sleep(5)
#             print("Token reset. Continuing...")
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {"role": "user", "content": f"{prompt}"}
            ],
            max_tokens=2048  # Adjust as needed
        )
        output = response.choices[0].message.content
#         lines = output.split('\n')
#         # Remove the first and last lines
#         modified_content = '\n'.join(lines[1:-1])
#         return modified_content
        return output
    except Exception as e:
        print("Error during API call:", e)
        return None

def process_file(file_path, target_directory, api_key, few_shot, CoT):
    description_path = 'data/description/' + file_path.split('/')[-1].split('_')[0].split('.')[0] + '.txt'
    # if the description file is not found, skip this file
    try:
        with open(description_path, 'r') as file:
            description = file.read()
    except:
        print("Description file not found:", description_path)
        return None
    
    with open(file_path, 'r') as file:
        code_content = file.read()

    debugged_content = debug_code_with_gpt(description, code_content, api_key, few_shot, CoT)

    if debugged_content:
        target_file_path = os.path.join(target_directory, os.path.basename(file_path))
        with open(target_file_path, 'w') as file:
            file.write(debugged_content)
#         print(f"Debugged code written to {target_file_path}")

def process_files_in_parallel(source_directory, target_directory, api_key, few_shot, CoT, max_workers=3):
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    file_paths = glob.glob(os.path.join(source_directory, '*.java'))
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_file, file_path, target_directory, api_key, few_shot, CoT)\
                   for file_path in file_paths]
        for future in tqdm(as_completed(futures), total=len(file_paths)):
            future.result()  # This will re-raise any exception caught during process_file execution

In [4]:
def main(few_shot, CoT):
    with open('API_KEY.txt', 'r') as file:
        api_key = file.read().strip()
    source_dir = 'data/formatted/buggy_codes_GPT'  # Source directory
#     source_dir = 'data/test/bug_codes_tests' # Source dir for pilot test
    target_dir = f'data/formatted/debugged_desc_GPTBuggyCodes_{few_shot}_{CoT}'  # Target directory
#     target_dir = f'data/test/debugged_GPTBuggyCodes_tests_{few_shot}_{CoT}' # Target dir for pilot test

    process_files_in_parallel(source_dir, target_dir, api_key, few_shot, CoT)

In [5]:
if __name__ == "__main__":
    # few_shot: '0' (zero-shot) or 'f' (few-shot)
    # CoT: 'n' (no Chain-of-Thought) or 'c' (with Chain-of-Thought)
    
    # RUN ALL COMBINATIONS
    few_shot = '0'
    CoT = 'n'
    main(few_shot, CoT)
    
    few_shot = '0'
    CoT = 'c'
    main(few_shot, CoT)
    
    few_shot = 'f'
    CoT = 'n'
    main(few_shot, CoT)
    
    few_shot = 'f'
    CoT = 'c'
    main(few_shot, CoT)

  3%|█▎                                         | 6/198 [00:11<05:06,  1.60s/it]

Description file not found: data/description/Count Number of Maximum BitwiseOR Subsets.txt


 36%|███████████████▎                          | 72/198 [01:52<02:21,  1.12s/it]

Description file not found: data/description/Maximum Count of Positive Integer and Negative Integer.txt


 69%|████████████████████████████▎            | 137/198 [03:18<01:15,  1.24s/it]

...

 85%|██████████████████████████████████▊      | 168/198 [04:23<00:39,  1.32s/it]

Description file not found: data/description/Find the Value of the Partition.txt


 98%|████████████████████████████████████████▏| 194/198 [05:00<00:05,  1.29s/it]

Description file not found: data/description/Maximum Number of Moves in a Grid.txt


100%|█████████████████████████████████████████| 198/198 [05:09<00:00,  1.56s/it]
